In [21]:
import pandas as pd,numpy as np, random,datetime

address_ds=pd.read_pickle(r'C:\Users\Administrator\实验代码\data\身份证号对应表.pkl')   
address_ds=address_ds.loc[address_ds['coordinate']=='success',]   
address_ds['num']=address_ds['num'].astype(str)
address_dict=dict(zip(address_ds['num'],address_ds['city_final']))#zip的作用是压缩元素级对象，生成新的列表

In [24]:
# 1.模拟生成业务数据


#basedata为基础申请表，产品为3000/6000额度，周期3/6期的现金分期业务，按期每月对应日期进行还款
class Calculate_data():
    def __init__(self):
        pass
    def cal_basedata(self,seed=123):
        np.random.seed(123)
        uer_data=pd.DataFrame({
                '订单号'  : ["id_"+"{x:0>5d}".format(x= _n) for _n in range(50000)],#列表推导式
                '申请期限': [random.randint(1, 100) for _ in range(50000)],
                '申请时间': [pd.to_datetime((datetime.datetime.now()-datetime.timedelta(days=random.randint(0, 180) )).strftime("%Y-%m-%d" ) )  for diff in range(50000)],
                '是否通过': [1 if random.randint(1, 100)>85 else 0 for _ in range(50000)],
                '身份证号': [str(random.choice(pd.unique(address_ds['num']) ))     for _ in range(50000)],
                } )
        uer_data['申请期限']=np.where(uer_data['申请期限']>95,3,6)
        uer_data['申请金额']=np.where(uer_data['申请期限']==6,6000,3000)
        uer_data['每期还款']=1000
        uer_data=uer_data[['订单号','申请时间','申请期限','申请金额','是否通过','身份证号','每期还款']]
        ##############################################################################
        repay_ds=uer_data.loc[uer_data['是否通过']==1,['订单号','申请期限','申请时间','每期还款' ]].reset_index(drop=True)
        repay_ds['ever_overdue']=[1 if random.randint(1, 100)<35 else 0 for _ in range(repay_ds.shape[0])]#列表推导式
        repay_ds['pnum_overdue']=[1 if random.randint(1, 100)<35 else 0 for _ in range(repay_ds.shape[0])]
        repay_data=pd.DataFrame()
        for item in repay_ds.itertuples():
            aa=pd.DataFrame({
                    'repaymentDay':[item.申请时间+ datetime.timedelta(days=diff *30) for diff in range(1,item.申请期限+1)] 
                    })
            if item.ever_overdue==0:
                aa['pnum_overdue']=0
            else:
                aa['pnum_overdue']=random.choice(range(1,item.申请期限+1)) 
            aa['申请时间']=item.申请时间
            aa['订单号']=item.订单号
            aa['申请期限']=item.申请期限
            aa['每期还款']=item.每期还款
            aa['申请期限_order']=range(1,item.申请期限+1)
            aa['ever_overdue']=item.ever_overdue
            aa['lateTime']=np.nan 
            days=random.randint(1, 365)   
            planState=1 if random.randint(1, 10)>=9   else 2
            for item2 in aa.itertuples():  
                if item2.pnum_overdue>= item2.申请期限_order:
                    aa.loc[aa.申请期限_order==item2.pnum_overdue,'lateTime']= np.nan  if (days- 30*item2.Index)<=0 else (days- 30*item2.Index)
                    aa.loc[item2.Index,'planState']=planState
                else:
                    aa.loc[item2.Index,'planState']=1
            repay_data=repay_data.append(aa,ignore_index=True)        

        repay_data['cutoff']=datetime.datetime.now().strftime("%Y-%m-%d")
        repay_data2=self.makeup_cutoff(indata=repay_data,cutdate='cutoff' )
        #申请期限_order:还款序号。第一次还款为1，第二次为2。一次类推
        #repaymentDay：应还款时间，时间格式。
        #lateTime：逾期时间天数。未逾期，为0
        #planState：是否还款。如果已还为1，未还为2.未到期为2。
        repay_data2=repay_data2[['订单号','申请期限','申请时间','每期还款' ,'申请期限_order','repaymentDay','lateTime','planState']]    
        return uer_data,repay_data2
    
    
# 2.数据处理

    def char2num(self,indata,varlist,address_dict=address_dict ):
        tmp=indata.copy()
        for var in varlist:
            tmp[var]=pd.to_numeric(tmp[var],  errors='coerce')
        return tmp

    def merge_ds(self,ds_list,on_var,how):
        for i in range(len(ds_list)):
            if i==1:
                tmp =pd.merge(ds_list[0],ds_list[1],on=on_var,how=how)
            elif i>1:
                tmp =pd.merge(tmp,ds_list[i],on=on_var,how=how)
        return tmp 

    def makeup_cutoff(self,indata,cutdate ):

        # 处理时间类型的数据
        test=indata.copy()
        #cutoff转换成时间格式
        test['cutdate']       =pd.to_datetime(indata[cutdate])
        #填补lateTime缺失值
        test['lateTime']      =test['lateTime'].fillna(0)
        test['lateTime']      =test['lateTime'].apply(lambda x :  np.timedelta64(int(x),'D')  ) 
        #计算真实还款时间
        test['real_date']     =test['repaymentDay'].values + test['lateTime'].values
        #还款时间和cutoff时间的先后顺序
        test['real_date_cut'] =test[['real_date','cutdate']] .min(axis=1)
        ################################################################################################################################
        #如果真实还款时间>cutoff时间，修改状态为2,修改逾期时间lateTime
        test['planState']     =np.where(test['real_date']>test['cutdate'],2,test['planState'])
        test['lateTime']      =(test['real_date_cut'].values-test['repaymentDay'].values)/np.timedelta64(1,'D')
        test.loc[test['lateTime']<0,'lateTime']      =0
        ################################################################################################################################
        return test

    
# 3.进件审批统计


    def random_city_data(self,indata,cutoff):
        #计算当前时间和申请时间的差值，用于计算近1周，2个月等时间段内的进件情况
        uer_data_tmp=indata.loc[ ((pd.to_datetime(datetime.datetime.now().strftime("%Y-%m-%d" ) )-indata['申请时间'])/np.timedelta64(1,'D'))<=cutoff,].reset_index(drop=True)
        #通过身份证前6位，得到具体地区。对应错误的进件排除
        uer_data_tmp['city']=uer_data_tmp['身份证号'].apply(lambda x: address_dict.get(str(x[:6]),None) )
        uer_data_tmp=uer_data_tmp.loc[uer_data_tmp['city'].notnull(),].reset_index(drop=True)
        #计算申请数apply_number，通过数pass_number
        tmp=uer_data_tmp.groupby(['city'])['是否通过'].agg([np.sum,pd.Series.count]).reset_index()
        tmp=tmp.rename(columns={'sum':'pass_number','count':'apply_number'})

    #        tmp['apply_ratio']=(tmp['apply_number']/sum(tmp['apply_number'])).apply(lambda x:'{x:.2f}'.format(x=x*100))
    #        tmp['pass_ratio']=(tmp['pass_number']/sum(tmp['pass_number'])).apply(lambda x:'{x:.2f}'.format(x=x*100))
    #        tmp['pass_rate']=(tmp['pass_number']/tmp['apply_number']).apply(lambda x:'{x:.2f}'.format(x=x*100))
        tmp['apply_ratio']=tmp['apply_number'].apply(lambda x: '{x:.2f}'.format(x=100*x/sum(tmp['apply_number'])) if sum(tmp['apply_number'])>0 else '0.00' )
        tmp['pass_ratio' ]=tmp['pass_number' ].apply(lambda x: '{x:.2f}'.format(x=100*x/sum(tmp['pass_number' ])) if sum(tmp['pass_number' ])>0 else '0.00' )
        tmp['pass_rate'  ]=tmp[['pass_number', 'apply_number']].apply(lambda x: '{x:.2f}'.format(x=100*x['pass_number']/x['apply_number'])  if  x['apply_number' ]>0 else '0.00' ,axis=1 )


        tmp=tmp.sort_values(by=['apply_number'],ascending=[0]).reset_index(drop=True)
        if tmp.shape[0]>10:
            tmp1=tmp.loc[:9,]
            tmp2=tmp.loc[10:,]
            last_row=pd.DataFrame({
                    'city':['其他'],
                    'pass_number':[sum(tmp2['pass_number'])] ,
                    'apply_number':[sum(tmp2['apply_number'])] ,
                    'apply_ratio':pd.to_numeric(['{x:.2f}'.format(x=  (sum(tmp2['apply_number'])/sum(tmp['apply_number' ]))*100 if sum(tmp['apply_number' ])>0 else 0 )]) ,
                    'pass_ratio' :pd.to_numeric(['{x:.2f}'.format(x=  (sum(tmp2['pass_number']) /sum(tmp['pass_number'  ]))*100 if sum(tmp['pass_number' ])>0 else 0)]),
                    'pass_rate'  :pd.to_numeric(['{x:.2f}'.format(x=  (sum(tmp2['pass_number']) /sum(tmp2['apply_number']))*100 if sum(tmp2['apply_number' ])>0 else 0)]),
                    })
            tmp1=tmp1.append(last_row,ignore_index=True)  
            tmp1=self.char2num( indata=tmp1, varlist=['apply_ratio','pass_ratio','pass_rate'] )
            tmp =self.char2num( indata=tmp , varlist=['apply_ratio','pass_ratio','pass_rate'] )
            return tmp[['city','pass_number','apply_number','apply_ratio','pass_ratio','pass_rate']] ,tmp1[['city','pass_number','apply_number','apply_ratio','pass_ratio','pass_rate']]
        else:
            return tmp[['city','pass_number','apply_number','apply_ratio','pass_ratio','pass_rate']] ,tmp[['city','pass_number','apply_number','apply_ratio','pass_ratio','pass_rate']]

    def _random_jinjian(self,indata ):
        _tmp=indata.copy()
        tmp_grp1=_tmp.groupby(['放款时间'])['申请金额'].sum().reset_index().rename(columns={'申请金额':'申请金额'})
        tmp_grp2=_tmp.groupby(['放款时间'])['身份证号'].agg(pd.Series.count).reset_index().rename(columns={'身份证号':'申请笔数'})
        tmp_grp3=_tmp.loc[_tmp['是否通过']==1,].groupby(['放款时间'])['申请金额'].sum().reset_index().rename(columns={'申请金额':'放款金额'})
        tmp_grp4=_tmp.loc[_tmp['是否通过']==1,].groupby(['放款时间'])['身份证号'].agg(pd.Series.count).reset_index().rename(columns={'身份证号':'放款笔数'})
        tmp_grp5=_tmp.loc[_tmp['是否通过']==1,].groupby(['放款时间'])['申请期限'].mean().reset_index().rename(columns={'申请期限':'放款平均期限'})
        tmp=self.merge_ds([tmp_grp1,tmp_grp2,tmp_grp3,tmp_grp4,tmp_grp5],on_var='放款时间',how='outer')

        tmp['申请件均']= tmp['申请金额']/tmp['申请笔数']
        tmp['申请件均']= tmp['申请件均'].astype(int)
        tmp['放款件均']= tmp['放款金额']/tmp['放款笔数']
        tmp['放款件均']= tmp['放款件均'].astype(int)

        tmp['通过率%']=tmp[['放款笔数','申请笔数']].apply(lambda x: '{x:.2f}'.format(x=100*x['放款笔数']/x['申请笔数'])  if x['申请笔数'] >0 else '0.00',axis=1 )
        tmp =self.char2num( indata=tmp , varlist=['通过率%'] )

        _tmp2=indata.sort_values(by=['身份证号','申请时间'] )
        _tmp2['贷款次数']=_tmp2.groupby(['身份证号']).cumcount()+1
        _tmp2=_tmp2.loc[_tmp2['贷款次数']>1,]

        tmp2_grp1=_tmp2.groupby(['放款时间'])['申请金额'].sum().reset_index().rename(columns={'申请金额':'复贷-申请金额'})
        tmp2_grp2=_tmp2.groupby(['放款时间'])['身份证号'].agg(pd.Series.count).reset_index().rename(columns={'身份证号':'复贷-申请笔数'})
        tmp2_grp3=_tmp2.loc[_tmp2['是否通过']==1,].groupby(['放款时间'])['申请金额'].sum().reset_index().rename(columns={'申请金额':'复贷-放款金额'})
        tmp2_grp4=_tmp2.loc[_tmp2['是否通过']==1,].groupby(['放款时间'])['身份证号'].agg(pd.Series.count).reset_index().rename(columns={'身份证号':'复贷-放款笔数'})
        tmp2 =self.merge_ds([tmp2_grp1,tmp2_grp2,tmp2_grp3,tmp2_grp4 ],on_var='放款时间',how='outer')

        tmp=pd.merge(tmp,tmp2,on='放款时间',how='outer')

        tmp['复贷-通过率%']=tmp[['复贷-放款笔数','复贷-申请笔数']].apply(lambda x: '{x:.2f}'.format(x=100*x['复贷-放款笔数']/x['复贷-申请笔数'])  if x['复贷-申请笔数'] >0 else '0.00',axis=1 )
        tmp['复贷通过占比']=tmp[['复贷-放款笔数','放款笔数'     ]].apply(lambda x: '{x:.2f}'.format(x=x['复贷-放款笔数']/x['放款笔数'])  if x['放款笔数'] >0 else '0.00',axis=1 )

        return tmp[["放款时间",'申请金额','申请笔数','放款金额','放款笔数','申请件均','放款件均','通过率%','放款平均期限',
                    '复贷-申请金额','复贷-申请笔数','复贷-放款金额','复贷-放款笔数','复贷通过占比','复贷-通过率%']]

    def random_jinjian_data(self,indata):
        ds1=indata.copy()
        ds2=indata.copy()

        week_cutday=[((datetime.datetime.now()-datetime.timedelta(days=diff ))).strftime("%Y-%m-%d" ) for diff in range(1,8) if  (datetime.datetime.now()-datetime.timedelta(days=diff )).weekday() ==0]
        #周
        ds1['放款时间']=ds1['申请时间'].apply(lambda x: x.strftime("%Y-%m-%d" ) if x>=pd.to_datetime(week_cutday[0]) else np.nan )
        tmp_1=self._random_jinjian( indata=ds1 )
        #为了格式整齐，防止某一天没有数据，补齐所有理论上的放款时间。
        format_week=pd.DataFrame({'放款时间': [(pd.to_datetime(week_cutday[0])+datetime.timedelta(days=diff)).strftime("%Y-%m-%d" ) for diff in range(0,7) if pd.to_datetime(week_cutday[0])+datetime.timedelta(days=diff)<=datetime.datetime.now()  ]  })
        tmp_1=pd.merge(tmp_1,format_week,on='放款时间',how='outer')
        tmp_1=tmp_1.fillna(0)

        #月
        ds2['放款时间']=ds2['申请时间'].apply(lambda x: x.strftime("%Y-%m") )
        tmp_2=self._random_jinjian( indata=ds2 )
        #为了格式整齐，防止某一天没有数据，补齐所有理论上的放款时间。
        format_month=pd.DataFrame({'放款时间': list(set([(ds2['申请时间'].min()+ datetime.timedelta(days=diff)).strftime("%Y-%m") for diff in range( int((datetime.datetime.now()-uer_data['申请时间'].min())/np.timedelta64(1,'D')) )]))  })
        tmp_2=pd.merge(tmp_2,format_month,on='放款时间',how='outer')
        tmp_2=tmp_2.fillna(0)

        return tmp_1.sort_values(['放款时间']),tmp_2.sort_values(['放款时间'])
    
    
    # 4.回款统计
    
    
    def random_huikuan_data(self,repay_ds,user_ds) :
        ds =repay_ds.copy()
        ds2=user_ds.copy()

        #到期天数
        ds_cal1=ds.groupby(['订单号'])['repaymentDay'].min().reset_index()
        ds_cal1['到期天数']=ds_cal1['repaymentDay'].apply(lambda x: int((datetime.datetime.now()-x)/np.timedelta64(1,'D')) )
        ds_cal1=ds_cal1[['订单号','到期天数']]

        #当前应还笔数,当前还款状态
        ds_cal2=ds.loc[ds['repaymentDay']<=pd.to_datetime(datetime.datetime.now().strftime("%Y-%m-%d")),].groupby('订单号')['申请期限_order'].max().reset_index()#当前应还笔数
        ds_cal2=pd.merge(ds_cal2, ds[['订单号', '申请期限_order','planState']],on=['订单号','申请期限_order'],how='left')#当前还款状态
        ds_cal2=ds_cal2.rename(columns={'申请期限_order':'当前应还笔数','planState':'当前还款状态'})

        #当前逾期天数
        ds['当前逾期天数']=np.where(ds['planState']==1,0, ds['lateTime'] ) #曾经逾期，如果还清，不计算
        ds_cal3=ds.groupby('订单号')['当前逾期天数'].max().reset_index()

        #当前逾期笔数
        ds_cal4=ds.loc[(ds['repaymentDay']<=pd.to_datetime(datetime.datetime.now().strftime("%Y-%m-%d"))) & (ds['planState'] ==2),].groupby('订单号')['申请期限_order'].count().reset_index().rename(columns={'申请期限_order':'当前逾期笔数'})
        ##############################################################################
        #订单号，到期天数，当前应还笔数，当前还款状态，前逾期天数，当前逾期笔数
        ds_cal=self.merge_ds( ds_list=[ds_cal1,ds_cal2,ds_cal3,ds_cal4],on_var='订单号',how='outer')
        ##############################################################################
        #订单号，到期天数，当前应还笔数，当前还款状态，前逾期天数，当前逾期笔数，申请期限，申请时间，每期还款
        ds_cal=pd.merge(ds_cal,ds2[['订单号' ,'申请期限','申请时间','每期还款']],on='订单号',how='left')
        ds_cal['放款时间']=ds_cal['申请时间'].apply(lambda x: x.strftime("%Y-%m") )
        ds_cal['当前应还笔数']=ds_cal['当前应还笔数'].fillna(0)
        ds_cal['当前还款状态']=ds_cal['当前还款状态'].fillna(2)
        ds_cal['当前逾期笔数']=ds_cal['当前逾期笔数'].fillna(0)

        #放款笔数
        cal1=ds_cal.groupby(['放款时间'])['订单号'].agg([pd.Series.nunique]).rename(columns={'nunique':'放款笔数'}).reset_index()

        #剩余金额和
        ds_cal['剩余金额和']=(ds_cal['申请期限']- ds_cal['当前应还笔数'] +  ds_cal['当前逾期笔数'])* ds_cal['每期还款']
        cal2=ds_cal.groupby(['放款时间'])['剩余金额和'].sum().reset_index()

        #笔数D10
        ds_cal['笔数D10']=np.where( (ds_cal['当前逾期天数']>=10) &  (ds_cal['当前逾期天数']<30),1,0)
        cal3=ds_cal.groupby(['放款时间'])['笔数D10'].sum().reset_index()

        #笔数D10_到期
        ds_cal['笔数D10_到期']=np.where(ds_cal['到期天数']>=10,1,0)
        cal4=ds_cal.groupby(['放款时间'])['笔数D10_到期'].sum().reset_index()

        #笔数M1
        ds_cal['笔数M1']=np.where((ds_cal['当前逾期天数']>=30) & (ds_cal['当前逾期天数']<60),1,0)
        cal5=ds_cal.groupby(['放款时间'])['笔数M1'].sum().reset_index()

        #笔数M1_到期
        ds_cal['笔数M1_到期']=np.where( ds_cal['到期天数']>=30 ,1,0)
        cal6=ds_cal.groupby(['放款时间'])['笔数M1_到期'].sum().reset_index()

        #笔数M2+
        ds_cal['笔数M2+']=np.where(ds_cal['当前逾期天数']>=60,1,0)
        cal7=ds_cal.groupby(['放款时间'])['笔数M2+'].sum().reset_index()

        #笔数M2+_到期
        ds_cal['笔数M2+_到期']=np.where(ds_cal['到期天数']>=60,1,0)
        cal8=ds_cal.groupby(['放款时间'])['笔数M2+_到期'].sum().reset_index()

        #金额D10
        ds_cal['金额D10']=np.where( (ds_cal['当前逾期天数']>=10) & (ds_cal['当前逾期天数']<30),ds_cal['剩余金额和'],0)
        cal9=ds_cal.groupby(['放款时间'])['金额D10'].sum().reset_index()

        #金额M1
        ds_cal['金额M1']=np.where( (ds_cal['当前逾期天数']>=30) & (ds_cal['当前逾期天数']<60),ds_cal['剩余金额和'],0)
        cal10=ds_cal.groupby(['放款时间'])['金额M1'].sum().reset_index()

        #金额M1
        ds_cal['金额M2+']=np.where( (ds_cal['当前逾期天数']>=60)  ,ds_cal['剩余金额和'],0)
        cal11=ds_cal.groupby(['放款时间'])['金额M2+'].sum().reset_index()

        tmp=self.merge_ds( ds_list=[cal1,cal2,cal3,cal4,cal5,cal6,cal7,cal8,cal9,cal10,cal11],on_var='放款时间',how='outer')

        tmp['笔数D10占比'] =tmp[['笔数D10','笔数D10_到期']].apply(lambda x: '{x:.2f}'.format(x=100*x['笔数D10']/x['笔数D10_到期']) if x['笔数D10_到期']>0 else '0' ,axis=1)
        tmp['笔数M1占比']  =tmp[['笔数M1' ,'笔数M1_到期' ]].apply(lambda x: '{x:.2f}'.format(x=100*x['笔数M1' ]/x['笔数M1_到期' ]) if x['笔数M1_到期' ]>0 else '0' ,axis=1)
        tmp['笔数M2+占比'] =tmp[['笔数M2+','笔数M2+_到期']].apply(lambda x: '{x:.2f}'.format(x=100*x['笔数M2+']/x['笔数M2+_到期']) if x['笔数M2+_到期']>0 else '0' ,axis=1)
        tmp['金额D10占比'] =tmp[['金额D10','剩余金额和'  ]].apply(lambda x: '{x:.2f}'.format(x=100*x['金额D10']/x['剩余金额和'  ]) if x['剩余金额和'  ]>0 else '0' ,axis=1)
        tmp['金额M1占比']  =tmp[['金额M1' ,'剩余金额和'  ]].apply(lambda x: '{x:.2f}'.format(x=100*x['金额M1'] /x['剩余金额和'  ]) if x['剩余金额和'  ]>0 else '0' ,axis=1)
        tmp['金额M2+占比'] =tmp[['金额M2+','剩余金额和'  ]].apply(lambda x: '{x:.2f}'.format(x=100*x['金额M2+']/x['剩余金额和'  ]) if x['剩余金额和'  ]>0 else '0' ,axis=1)

    #        (tmp['笔数D10']/tmp['放款笔数']).apply(lambda x: '{x:.2f}'.format(x=100*x))
    #        tmp['笔数M1占比']  =(tmp['笔数M1']/tmp['放款笔数']).apply(lambda x: '{x:.2f}'.format(x=100*x))
    #        tmp['笔数M2+占比'] =(tmp['笔数M2+']/tmp['放款笔数']).apply(lambda x: '{x:.2f}'.format(x=100*x))
    #        tmp['金额D10占比'] =(tmp['金额D10']/tmp['剩余金额和']).apply(lambda x: '{x:.2f}'.format(x=100*x))
    #        tmp['金额M1占比']  =(tmp['金额M1']/tmp['剩余金额和']).apply(lambda x: '{x:.2f}'.format(x=100*x))
    #        tmp['金额M2+占比'] =(tmp['金额M2+']/tmp['剩余金额和']).apply(lambda x: '{x:.2f}'.format(x=100*x))
        tmp=tmp[['放款时间','放款笔数','剩余金额和','笔数D10','笔数D10占比','笔数M1','笔数M1占比','笔数M2+','笔数M2+占比','金额D10','金额D10占比','金额M1','金额M1占比','金额M2+','金额M2+占比',
                 '笔数D10_到期','笔数M1_到期','笔数M2+_到期']]
        tmp=self.char2num( indata=tmp, varlist= ['笔数D10占比','笔数M1占比','笔数M2+占比','金额D10占比','金额M1占比','金额M2+占比'])

        #为了格式整齐，防止某一天没有数据，补齐所有理论上的放款时间。
        format_month=pd.DataFrame({'放款时间': list(set([(ds['申请时间'].min()+ datetime.timedelta(days=diff)).strftime("%Y-%m") for diff in range( int((datetime.datetime.now()-uer_data['申请时间'].min())/np.timedelta64(1,'D')) )]))  })
        tmp=pd.merge(tmp,format_month,on='放款时间',how='outer')
        tmp=tmp.fillna(0)

        return tmp

    def random_yuqilv_data(self,indata,prefix=''):

        tmp=indata.copy()
        tmp=pd.merge(tmp,tmp.groupby(['订单号'])['repaymentDay'].min().reset_index().rename(columns={'repaymentDay':'first_date'}),on='订单号',how='left')
        tmp['放款时间']=tmp['申请时间'].apply(lambda x: x.strftime("%Y-%m") )
        tmp['cutoff']=tmp['first_date'].apply(lambda x:  ( x + datetime.timedelta(days=30)).strftime("%Y-%m-%d" ) )

        tmp2=self.makeup_cutoff( indata=tmp,cutdate='cutoff' )
        ##############################################################################
        #到期期数
        ds_cal1=tmp.groupby(['订单号','放款时间'])['repaymentDay'].min().reset_index()
        ds_cal1['到期天数']=ds_cal1['repaymentDay'].apply(lambda x: int((datetime.datetime.now()-x)/np.timedelta64(1,'D')) )
        ds_cal1=ds_cal1[['订单号','到期天数']]
        ##############################################################################
        ds_cal2=tmp2.groupby(['订单号','放款时间'])['lateTime'].max().reset_index()
        ##############################################################################
        ds_cal3=pd.merge(ds_cal2,ds_cal1,on='订单号',how='outer')
        #
        ds_cal3['笔数到期']=np.where(ds_cal3['到期天数']>=30, 1,0)

        ds_cal3['笔数首逾D10+']=np.where(ds_cal3['lateTime']>=10, 1,0)
        ds_cal3['笔数首逾M1+']=np.where(ds_cal3['lateTime']>=30, 1,0)

        tmp_tot=ds_cal3.groupby(['放款时间'])['笔数到期','笔数首逾D10+','笔数首逾M1+'].sum().reset_index()
        ##############################################################################
        #放款笔数
        ds_cal4=tmp.groupby(['放款时间'])['订单号'].agg([pd.Series.nunique]).rename(columns={'nunique':'放款笔数'}).reset_index()
        ##############################################################################
        tmp_tot=pd.merge(tmp_tot,ds_cal4,on='放款时间',how='outer')

        tmp_tot['笔数首逾D10+占比']=(tmp_tot['笔数首逾D10+']/tmp_tot['笔数到期']).apply(lambda x: '{x:.2f}'.format(x=100*x))
        tmp_tot['笔数首逾M1+占比']=(tmp_tot['笔数首逾M1+']/tmp_tot['笔数到期']).apply(lambda x: '{x:.2f}'.format(x=100*x))
        tmp_tot=self.char2num( indata=tmp_tot, varlist= ['笔数首逾D10+占比','笔数首逾M1+占比'])
        tmp_tot=tmp_tot[["放款时间",'放款笔数','笔数到期','笔数首逾D10+', '笔数首逾M1+','笔数首逾D10+占比','笔数首逾M1+占比']]

        tmp_tot.columns=["放款时间"]+[prefix+name for name in ['放款笔数','笔数到期','笔数首逾D10+', '笔数首逾M1+','笔数首逾D10+占比','笔数首逾M1+占比']]

        return tmp_tot   
    
    
  # 5.逾期统计


    def random_migration_data(self,indata):
        ds=indata.copy()
        ds['放款时间']=ds['申请时间'].apply(lambda x: x.strftime("%Y-%m") )
        ds['cutdate_1']=datetime.datetime.now().strftime("%Y-%m" )+'-01'
        ds['cutdate_2']=datetime.datetime.now().strftime("%Y-%m-%d" ) 
        ds=ds.loc[ds['申请时间']<=pd.to_datetime(datetime.datetime.now().strftime("%Y-%m" )+'-01'),]
        ds1=self.makeup_cutoff( indata=ds,cutdate='cutdate_1' )
        ds2=self.makeup_cutoff( indata=ds,cutdate='cutdate_2' )

        #前逾期天数
        ds1['当前逾期天数']=np.where(ds1['planState']==1,0, ds1['lateTime'] ) 
        ds1_cal1=ds1.groupby(['订单号','放款时间'])['当前逾期天数'].max().reset_index()

        ds2['当前逾期天数']=np.where(ds2['planState']==1,0, ds2['lateTime'] ) 
        ds2_cal1=ds2.groupby(['订单号','放款时间'])['当前逾期天数'].max().reset_index()

        ds_cal1=pd.merge(ds1_cal1.rename(columns={'当前逾期天数':'月初_当前逾期天数'}),ds2_cal1[['订单号','当前逾期天数']].rename(columns={'当前逾期天数':'当天_当前逾期天数'}),on='订单号',how='outer' )
        ds_cal1['repay_number']=1
        ds_cal1['c_start'] =np.where(ds_cal1['月初_当前逾期天数']==0,1,0)
        ds_cal1['m1_start']=np.where((ds_cal1['月初_当前逾期天数']<=30) & (ds_cal1['月初_当前逾期天数']>0),1,0)
        ds_cal1['m2_start']=np.where((ds_cal1['月初_当前逾期天数']<=60) & (ds_cal1['月初_当前逾期天数']>30),1,0)
        ds_cal1['m2+_start']=np.where((ds_cal1['月初_当前逾期天数']>60),1,0)

        ds_cal1['c_m1']  =np.where((ds_cal1['c_start'] ==1) & (ds_cal1['当天_当前逾期天数']>0)  ,1,0)
        ds_cal1['m1_m2'] =np.where((ds_cal1['m1_start']==1) & (ds_cal1['当天_当前逾期天数']>30) ,1,0)
        ds_cal1['m2_m2+']=np.where((ds_cal1['m2_start']==1) & (ds_cal1['当天_当前逾期天数']>60) ,1,0)

        tmp=ds_cal1.groupby('放款时间')['repay_number','c_start','m1_start','m2_start','m2+_start','c_m1','m1_m2','m2_m2+'].sum().reset_index()

        tmp['c_m1_rate'] = ( tmp['c_m1'] /tmp['c_start']  ).apply(lambda x: '{x:.2f}'.format(x=100*x))
        tmp['m1_m2_rate'] = ( tmp['m1_m2'] /tmp['m1_start']  ).apply(lambda x: '{x:.2f}'.format(x=100*x))
        tmp['m2_m2+_rate'] = ( tmp['m2_m2+'] /tmp['m2_start']  ).apply(lambda x: '{x:.2f}'.format(x=100*x))
        tmp=self.char2num( indata=tmp, varlist= ['c_m1_rate','m1_m2_rate','m2_m2+_rate' ])

        return tmp[["放款时间",'repay_number','c_start','m1_start','m2_start','m2+_start','c_m1','c_m1_rate','m1_m2','m1_m2_rate','m2_m2+','m2_m2+_rate']]    

    def random_city_yuqi_data(self,repay_ds,user_ds,cutoff=30 ):
        uer_data_tmp=pd.merge(repay_ds,user_ds[['订单号','身份证号' ]],on='订单号',how='left')
        uer_data_tmp=uer_data_tmp.loc[((pd.to_datetime(datetime.datetime.now().strftime("%Y-%m-%d" ) )-uer_data_tmp['申请时间'])/np.timedelta64(1,'D')) <=cutoff, ].reset_index(drop=True)
        uer_data_tmp['city']=uer_data_tmp['身份证号'].apply(lambda x: address_dict.get(str(x[:6]),None) )
        uer_data_tmp=uer_data_tmp.loc[uer_data_tmp['city'].notnull(),].reset_index(drop=True)

        repay_cutoff=uer_data_tmp.groupby(['订单号','city'])['repaymentDay'].min().reset_index()
        repay_cutoff['cutoff_date']=repay_cutoff['repaymentDay']+ datetime.timedelta(days= 30)
        repay_cutoff=repay_cutoff.loc[repay_cutoff['cutoff_date']<=datetime.datetime.now(),]

        uer_data_tmp=pd.merge(uer_data_tmp,repay_cutoff[['订单号','cutoff_date']],on='订单号',how='inner')
        uer_data_tmp=self.makeup_cutoff(uer_data_tmp,'cutoff_date')
        tmp1=repay_cutoff.groupby('city')['订单号'].count().reset_index().rename(columns={'订单号':'首逾到期'})
        tmp2=uer_data_tmp.groupby(['订单号','city'])['lateTime'].max().reset_index()
        tmp2['首逾D10+']=np.where(tmp2['lateTime']>=10,1,0)
        tmp2['首逾M1+'] =np.where(tmp2['lateTime']>=30,1,0)
        tmp2=tmp2.groupby('city')['首逾D10+','首逾M1+'].sum().reset_index()

        tmp=pd.merge(tmp1,tmp2,on='city',how='outer')
        tmp['首逾D10+率']=(tmp['首逾D10+']/tmp['首逾到期']).apply(lambda x: '{x:.2f}'.format(x=100*x))
        tmp['首逾M1+率']=(tmp['首逾M1+']/tmp['首逾到期']).apply(lambda x: '{x:.2f}'.format(x=100*x))

        tmp=tmp.sort_values(by=['首逾D10+率'],ascending=[0]).reset_index(drop=True)

        if tmp.shape[0]>10:
            tmp1=tmp.loc[:9,]
            tmp2=tmp.loc[10:,]
            last_row=pd.DataFrame({
                    'city':['其他'],
                    '首逾到期':[sum(tmp2['首逾到期'])] ,
                    '首逾D10+':[sum(tmp2['首逾D10+'])] ,
                    '首逾M1+':[sum(tmp2['首逾M1+'])] ,
                    '首逾D10+率':pd.to_numeric(['{x:.2f}'.format(x= 100* (sum(tmp2['首逾D10+'])/sum(tmp2['首逾到期'])))   ] ),
                    '首逾M1+率' :pd.to_numeric(['{x:.2f}'.format(x= 100* (sum(tmp2['首逾M1+'])/sum(tmp2['首逾到期'])) ) ]) ,
                    })
            tmp1=tmp1.append(last_row,ignore_index=True)        
            tmp1=self.char2num( indata=tmp1, varlist= ['首逾D10+率','首逾M1+率'])
            tmp =self.char2num( indata=tmp , varlist= ['首逾D10+率','首逾M1+率'])

        return tmp[['city','首逾到期','首逾D10+','首逾M1+','首逾D10+率','首逾M1+率']] ,tmp1 [['city','首逾到期','首逾D10+','首逾M1+','首逾D10+率','首逾M1+率']] 

In [26]:
# 6.main

if __name__ == "__main__":
    cd=Calculate_data()
    uer_data,repay_data=cd.cal_basedata(seed=1234)
    user_ds_fd=uer_data.sort_values(by=['身份证号','申请时间'] )
    user_ds_fd['贷款次数']=user_ds_fd.groupby(['身份证号']).cumcount()+1
    user_ds_fd=user_ds_fd.loc[user_ds_fd['贷款次数']>1,].reset_index(drop=True)
    city_apply_day,city_apply_day1       =cd.random_city_data(indata=uer_data,cutoff=0 )
    city_apply_week,city_apply_week1     =cd.random_city_data(indata=uer_data,cutoff=7 )
    city_apply_1month,city_apply_1month1 =cd.random_city_data(indata=uer_data,cutoff=30 )
    city_apply_6month,city_apply_6month1 =cd.random_city_data(indata=uer_data,cutoff=180 )   
    jinjian_week,jinjian_month=cd.random_jinjian_data(indata =uer_data)
    huikuan_month=cd.random_huikuan_data(repay_ds=repay_data,user_ds=uer_data)
    yuqilv_month   =cd.random_yuqilv_data(indata=repay_data ,prefix='')
    yuqilv_month_fd=cd.random_yuqilv_data(indata=repay_data.loc[repay_data['订单号'].isin(user_ds_fd['订单号']),].reset_index(drop=True),prefix='复贷_')
    
    migration_month=cd.random_migration_data(indata=repay_data)
    city_yuqi_3month,city_yuqi_3month2 =cd.random_city_yuqi_data(repay_ds=repay_data,user_ds=uer_data,cutoff=90)
    city_yuqi_6month,city_yuqi_6month2 =cd.random_city_yuqi_data(repay_ds=repay_data,user_ds=uer_data,cutoff=180)